In [ ]:
import numpy as np
import pandemic as pan
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
keys = ['initial_infected_fraction', 'initial_removed_fraction', 'incubation_period', 
        'serial_interval', 'symptomatic_fraction', 'recovery_days', 'quarantine_days', 
        'days_indetectable', 'R0', 'contact_rate', 'npi_factor', 'contact_tracing_testing_rate', 
        'contact_tracing_quarantine_rate', 'contact_tracing_days', 'daily_testing_fraction', 
        'daily_testing_false_positive', 'daily_testing_false_negative', 'class_size_limit', 
        'contact_upscale_factor', 'friendship_contacts', 'academic_contacts', 'broad_social_contacts', 
        'department_environmental_contacts', 'broad_environmental_contacts', 'residential_neighbors', 
        'online_transition']

keys_to_round = ['incubation_period', 'serial_interval', 'recovery_days',
                 'quarantine_days', 'days_indetectable', 'contact_tracing_days',
                 'class_size_limit', 'friendship_contacts', 'academic_contacts',
                 'broad_social_contacts', 'department_environmental_contacts',
                 'broad_environmental_contacts', 'residential_neighbors', 'online_transition']

These are the best values I've found

In [ ]:
map_train = {'R0': 4.864972,
 'academic_contacts': 1.9999999,
 'broad_environmental_contacts': 4.0,
 'broad_social_contacts': 1.0,
 'class_size_limit': 147.0,
 'contact_rate': 8.86977,
 'contact_tracing_days': 1.0,
 'contact_tracing_quarantine_rate': 1.6692917,
 'contact_tracing_testing_rate': 0.89044154,
 'contact_upscale_factor': 1.1708736,
 'daily_testing_false_negative': 0.030860264,
 'daily_testing_false_positive': 0.0009862408,
 'daily_testing_fraction': 0.00036885968,
 'days_indetectable': 2.0,
 'department_environmental_contacts': 5.0,
 'friendship_contacts': 4.0,
 'incubation_period': 6.0,
 'initial_infected_fraction': 0.0036397844,
 'initial_removed_fraction': 0.038633253,
 'npi_factor': 0.46783707,
 'online_transition': 75.0,
 'quarantine_days': 21.0,
 'recovery_days': 12.0,
 'residential_neighbors': 1.0,
 'serial_interval': 9.0,
 'symptomatic_fraction': 0.22690229}

Need to add a couple parameters and scenario name.  Also, need to do some rounding.

In [ ]:
map_train['scenario_name'] = 'trial_optimum'
map_train['quarantining'] = 1
map_train['social_distancing'] = 1
map_train['contact_tracing'] = 1
for k in keys_to_round:
    map_train[k] = int(np.round(map_train[k]))

Initialize pandemic object

In [ ]:
pandemic = pan.Disease(map_train)

In [ ]:
recorder = pan.analysis.recorder(['tests_performed_total', 'positive_tests_total', 'active_cases'], 
                                     './trial_optimum.h5')

Run a handful of simulations based on params above.

In [ ]:
pandemic.multiple_runs(10,recorder)

In [ ]:
tests_total = recorder.values['tests_performed_total']['all_runs']
pos_total = recorder.values['positive_tests_total']['all_runs']
act_total = recorder.values['active_cases']['all_runs']

Import data for plotting.

In [ ]:
data = pd.read_csv('datasets/testing.csv',skiprows=9)
active = pd.read_csv('datasets/active.csv',skiprows=1)
av = active.values[7:,2]
r = np.linspace(0,len(av)-1,len(av))
dd = np.vstack((r,av)).T
ac = dd[np.invert(np.isnan(dd[:,1].astype(float)))]

rapid_pcr_tests = np.nan_to_num(data.values[79:,2].astype(float))
rapid_pcr_pos = np.nan_to_num(data.values[79:,4].astype(float))
rapid_antigen_tests = np.nan_to_num(data.values[79:,3].astype(float))
rapid_antigen_pos = np.nan_to_num(data.values[79:,5].astype(float))
state_tests = np.nan_to_num(data.values[79:,6].astype(float))
state_pos = np.nan_to_num(data.values[79:,8].astype(float))

total_tests = rapid_pcr_tests + rapid_antigen_tests + state_tests
total_pos = rapid_pcr_pos + rapid_antigen_pos + state_pos
cum_tests = np.cumsum(total_tests)
cum_pos = np.cumsum(total_pos)
n_data = 194 - 90

Plot

In [ ]:
fig,axs = plt.subplots(nrows=3,sharex=True)
fig.set_size_inches(12,12)
[axs[0].plot(pos_total[i],'b-',alpha=0.3) for i in range(10)]
axs[0].plot(cum_pos[:pos_total.shape[1]],'r-')
axs[0].set_xlabel('day')
axs[0].set_ylabel('Cumul. pos.')
[axs[1].plot(tests_total[i],'b-',alpha=0.3) for i in range(10)]
axs[1].plot(cum_tests[:pos_total.shape[1]],'r-')
axs[1].set_xlabel('day')
axs[1].set_ylabel('Cumul. tests')
fig.savefig('abm_vs_data.pdf')
[axs[2].plot(act_total[i],'b-',alpha=0.3) for i in range(10)]
axs[2].set_xlabel('day')
axs[2].set_ylabel('Active cases')